In [0]:
#write by zhr_momeni
#split merged data into test and train set with ratio 3:7
import csv
import numpy as np
import pandas as pd

def SplitToTrainTest():
    data = pd.read_csv("MergedDs.csv",header = 0,sep = ',') 
    TestData = data.iloc[::4,:]
    esm = list()
    esm.append(data)
    esm.append(TestData)  

    df = pd.concat(esm)

    TrainData = df.drop_duplicates(keep = False)
    TrainData.to_csv('Train.csv', index = None, header=True)
    TestData.to_csv('Test.csv', index = None, header=True)
    return "done"
print SplitToTrainTest()


done


In [0]:
#Add an ageClass as label to each sample based on the following three age ranges:
import numpy as np
import pandas as pd

def getYData(sourceName,destinationName):
    data = pd.read_csv(sourceName,header = 0,sep = ',')   
    y = data.loc[:, "lable"]
    multiClass = list()
    for i in range(len(y)):
        
        if y[i]>=0.0 and y[i]<=20.0:
            multiClass.append(0)
        
        if y[i]>20.0 and y[i]<=50.0:
            multiClass.append(1)
        
        if y[i]>50.0 and y[i]<=103.0:
            multiClass.append(2)
        
    
    data['class'] = pd.Series(multiClass, index=data.index)
    data.to_csv(destinationName, index = None, header=True)
    return "done"

print getYData("Train.csv","relabeldTrain.csv")  
print getYData("Test.csv","relabeldTest.csv")


done


In [0]:
# Confiusion matrix
import pandas as pd
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.metrics import confusion_matrix

def getData():
    data = pd.read_csv("relabeldTrain.csv",header = 0,sep = ',')   
    y = data.loc[:,"class"]
    x = data.drop(["lable","class"], axis=1)
    return x,y

X , Y = getData()

pca = PCA(n_components=60)
x_new = pca.fit_transform(X,Y)

clf = svm.SVC(kernel = "linear", gamma="auto", C= 931.208345808, degree=1, shrinking = True, probability = False  , decision_function_shape='ovo')
score = cross_val_score(clf, x_new, Y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))
pred = cross_val_predict(clf, x_new, Y, cv=5)
label = [1,2,3] #or for 10 class: [1,2,3,4,5,6,7,8,9,10]
print confusion_matrix(Y, pred,labels=label)


In [0]:
#The last thing to do in the first step
#Labeling test data based on three age ranges found.
import pandas as pd
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.metrics import confusion_matrix

def getData(fileName):
    data = pd.read_csv(fileName,header = 0,sep = ',')   
    y = data.loc[:,"class"]
    x = data.drop(["lable","class"], axis=1)
    return x,y

X , Y = getData("relabeldTrain.csv")
Xt , Yt = getData("relabeldTest.csv")

pca = PCA(n_components=60)
x_new = pca.fit_transform(X,Y)
xt_new = pca.transform(Xt)


clf = svm.SVC(kernel = "linear", gamma="auto", C= 931.208345808, degree=1, shrinking = True, probability = False  , decision_function_shape='ovo')
clf.fit(x_new, Y)


def writeData(fileName,arr):
    data = pd.read_csv(fileName,header = 0,sep = ',')   
    data['prediction'] = pd.Series(arr, index=data.index)
    data.to_csv('relabeldTest.csv', index = None, header=True)
    return "done!"

pred = clf.predict(xt_new)
print confusion_matrix(Yt, pred,labels=[0,1,2])
print writeData("relabeldTest.csv",pred)